In [93]:
# Importing Google Sheets and json packages

import json
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

# Using the google api to access google sheets
scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]

# Credentials created from Google API for google sheets
creds = Credentials.from_service_account_file("Credentials.json", scopes=scopes)
client = gspread.authorize(creds)

# Enter the sheet ID from the url of your sheet. Hidden for security reasons
sheet_id = """Enter your Sheet ID"""
sheet = client.open_by_key(sheet_id)

In [95]:
# Exporting the existing fantasy player names and stat headers from the sheet
values_list = sheet.sheet1.get_all_values()
header = values_list[0]
pd.DataFrame(values_list[1:], columns=header)

,Player Name,Minutes played,Goals,Assists,Goals conceded,Saves,Penalties won,Penalties conceded,Card Type,FotMob rating,Position,Fantasy Points
0,Erling Haaland,,,,,,,,,,ST,0
1,Julian Alvarez,,,,,,,,,,ST,0
2,Hugo Bueno,,,,,,,,,,LB,0
3,Boubacar Traore,,,,,,,,,,CDM,0
4,Mario Lemina,,,,,,,,,,CDM,0
5,Nathan Ake,,,,,,,,,,CB,0
6,Josko Gvardiol,,,,,,,,,,CB,0
7,Rodri,,,,,,,,,,CDM,0
8,Mateo Kovacic,,,,,,,,,,CM,0
9,Joao Gomes,,,,,,,,,,CDM,0


In [73]:
# Importing Chromedriver Packages
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Setting up the logs for Chrome browser
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)

# Setting the path for the driver exec file
DRIVER_PATH = 'C:/Users/bornf/Documents/Python Scripts/fotmob-scraper/chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Opening the url of the match that we are interested in
driver.get("https://www.fotmob.com/matches/manchester-city-vs-wolverhampton-wanderers/2emo8d#4193889:tab=lineup")

C:\Users\bornf\AppData\Local\Temp\ipykernel_14180\1932991813.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [74]:
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display
import re
from unidecode import unidecode

# Wait for the page to load
wait = WebDriverWait(driver, 10)

# Lists for holding players' individual stats
player_dataframes = []
player_names = []

try:
    # Using domain knowledge, we create a pattern from the game url to locate the players' stats
    pattern = "/matches/manchester-city-vs-wolverhampton-wanderers/2emo8d?player="
    
    # Locate all <a> elements with href containing the pattern
    links = wait.until(EC.presence_of_all_elements_located((By.XPATH, f'//a[contains(@href, "{pattern}")]')))
    links = list(set(links))

    # Iterate over the links and click each one
    for index, link in enumerate(links):
        
        try:
            # Scroll into view if necessary
            driver.execute_script("arguments[0].scrollIntoView();", link)

            # Click the link
            driver.execute_script("arguments[0].click();", link)
            print(f"Clicked link {index + 1}: {link.get_attribute('href')}")

            # Obtain new link
            new_link = link.get_attribute('href')

            # Nest the process to obtain HTML from the new link
            new_driver = webdriver.Chrome(executable_path=DRIVER_PATH)
            new_driver.get(new_link)

            # New wait to scrape the data using the CSS selector which is standard across Fotmob
            new_wait = WebDriverWait(new_driver, 10)
            target_element = new_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.css-1c7p1w5')))
            html_data = target_element.get_attribute('outerHTML')
            
            # Parse HTML data with BeautifulSoup
            soup = BeautifulSoup(html_data, 'html.parser')
            
            # Using the selector again to pull the player name, which is standard across Fotmob
            player_element = new_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.css-112ye3e')))
            player_name_data = player_element.get_attribute('outerHTML')
            pl_soup = BeautifulSoup(player_name_data, 'html.parser')
            
            # Find the 'a' tag
            a_tag = pl_soup.find('a', class_='css-112ye3e')

            # Extract the player name
            player_name = unidecode(a_tag.find('div', class_='css-10x6lqx-PlayerName e1jy3r7k1').text)
            
            # Find all <ul> elements which store stats by categories
            ul_elements = soup.find_all('ul', class_='css-1szro0h-StatSection e1jy3r7k2')

            # List to hold dataframes
            dataframe = pd.DataFrame([['Name', player_name]], columns=['Stat Title', 'Stat Value'])
            player_names.append(player_name)

            # Parse through the categories and store the stats from each group
            for ul in ul_elements:
                
                # Extract data for each <ul> using the css selector
#                 pattern = r'PlayerRatingStyled'
                data = []
                for li in ul.find_all('li', class_='css-qc4i8b-RowContainer e1jy3r7k3'):
                    stat_title = li.find('span', class_='StatTitle')
                    stat_value = li.find('span', class_='StatValue')
                    # Stats are held in weird div tags where the stat title and corresponding stat value are held in 
                    # Consecutive tags but not together. So to overcome that I have written the code below that checks
                    # If the stat title is not empty, and the value is not empty, add the value else check the next div tag
                    if stat_title:
                        title = stat_title.get_text(strip=True)
                        value = stat_value.get_text(strip=True) if stat_value else li.find('div').find('span').get_text(strip=True)
                        data.append([title, value])

                # Convert to DataFrame
                df = pd.DataFrame(data, columns=['Stat Title', 'Stat Value'])
                dataframe = pd.concat([dataframe, df], ignore_index=True)

            # `dataframes` now contains the DataFrames for each <ul>
            display(dataframe)
            player_dataframes.append(dataframe)

    #             df = pd.read_html(html_data)[0]
    #             print(df)
        except Exception as e:
            print(f"Could not click link {index + 1}: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Clicked link 1: https://www.fotmob.com/matches/manchester-city-vs-wolverhampton-wanderers/2emo8d?player=955529#4193889:tab=lineup


C:\Users\bornf\AppData\Local\Temp\ipykernel_14180\725649700.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  new_driver = webdriver.Chrome(executable_path=DRIVER_PATH)


,Stat Title,Stat Value
0,Name,Matheus Nunes
1,FotMob rating,6.1
2,Minutes played,11
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,8/8 (100%)
7,Chances created,0
8,Expected assists (xA),0.03
9,xG + xA,0.03


Clicked link 2: https://www.fotmob.com/matches/manchester-city-vs-wolverhampton-wanderers/2emo8d?player=865931#4193889:tab=lineup


C:\Users\bornf\AppData\Local\Temp\ipykernel_14180\725649700.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  new_driver = webdriver.Chrome(executable_path=DRIVER_PATH)


...

C:\Users\bornf\AppData\Local\Temp\ipykernel_14180\725649700.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  new_driver = webdriver.Chrome(executable_path=DRIVER_PATH)


,Stat Title,Stat Value
0,Name,Mario Lemina
1,FotMob rating,6.6
2,Minutes played,45
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,29/33 (88%)
7,Chances created,0
8,Touches,38
9,Successful dribbles,1/1 (100%)


Clicked link 43: https://www.fotmob.com/matches/manchester-city-vs-wolverhampton-wanderers/2emo8d?player=240054#4193889:tab=lineup


C:\Users\bornf\AppData\Local\Temp\ipykernel_14180\725649700.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  new_driver = webdriver.Chrome(executable_path=DRIVER_PATH)


,Stat Title,Stat Value
0,Name,Pablo Sarabia
1,FotMob rating,6.6
2,Minutes played,14
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,9/14 (64%)
7,Chances created,0
8,Expected goals (xG),0.43
9,Expected goals on target (xGOT),0.00


In [75]:
# This part of the code is to get disciplinary data from WhoScored

# Re-initializing the WebDriver
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.set_page_load_timeout(10)

# Opening the webpage
driver.get('https://www.whoscored.com/Matches/1729372/Live/England-Premier-League-2023-2024-Manchester-City-Wolves')

# Creating a WebDriverWait object
wait = WebDriverWait(driver, 10)

# Waiting until the element with ID 'live-incidents' is present
table_element = wait.until(EC.presence_of_element_located((By.ID, 'live-incidents')))

# Getting the HTML of the element
table_html = table_element.get_attribute('outerHTML')

# Parsing HTML data with BeautifulSoup
soup = BeautifulSoup(table_html, 'html.parser')

# Definitions from WhoScored's data for yellow and red cards
attribute_name = 'data-type'
desired_value = '17'
attribute_to_retrieve = 'data-card-type'

# Finding all the card data by using attribute name and value
elements_with_attribute = soup.find_all(lambda tag: tag.get(attribute_name) == desired_value)

# Initializing a list to hold all the card data
card_data_list = []

# Printing and storing the values of the desired attribute
for x in range(0,len(elements_with_attribute),2):
    # Retrieve the value of the other attribute
    element = elements_with_attribute[x]
    player_name = element.text
    element2 = elements_with_attribute[x+1]
    attribute_value = element2.get(attribute_to_retrieve)
    print(f"Tag: {attribute_name}: {element2.get(attribute_name)}, {attribute_to_retrieve}: {attribute_value}, Content: {element.text}")
    card_data_list.append([element.text, attribute_value])
    
driver.quit()

C:\Users\bornf\AppData\Local\Temp\ipykernel_14180\1275278032.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


Tag: data-type: 17, data-card-type: 31, Content: Mario Lemina
Tag: data-type: 17, data-card-type: 31, Content: Nélson Semedo
Tag: data-type: 17, data-card-type: 31, Content: Boubacar Traoré
Tag: data-type: 17, data-card-type: 31, Content: João Gomes


In [76]:
# Using fuzzywuzzy, we match the player names from the card data list to the original dataframes, since there are
# accents on the name that will not allow us to do a direct string match
from fuzzywuzzy import fuzz
from unidecode import unidecode
import re

# While the card data list is not empty, we loop through the list and the dataframes to find 90% match. This logic will fail
# in case two players have the exact same names but that improvement is in the pipelines for later versions
while len(card_data_list) > 0:
    # We initialize n inside the loop as we will pop the item from the list and the next player will again be the 0th item
    n = 0
    cd = card_data_list[n]
    normalized_name1 = unidecode(cd[0])
    
    # Looping through the dataframes and extracting the player names
    for i in range(len(player_dataframes)):
        df = player_dataframes[i]
        name = df[df['Stat Title'] == 'Name']['Stat Value'].iloc[0]
        normalized_name2 = unidecode(name)
        
        # Matching the stats if the names are at least 90% similar
        if fuzz.ratio(normalized_name1, normalized_name2) >= 90:
            temp_df = pd.DataFrame([['Card Type', cd[1]]], columns=['Stat Title', 'Stat Value'])
            updated_df = pd.concat([df, temp_df], ignore_index=True)

            player_dataframes[i] = updated_df

            card_data_list.pop(n)
            n += 1
            break

In [77]:
# We can see that all players' data has been matched if the list is empty
card_data_list

[]

In [78]:
# Printing the dataframes again with the updated card details. A yellow card is type 31 and a red card is type 32 and two yellows are type 33. This will be used in our final fantasy points calculation
for df in player_dataframes:
    display(df)

,Stat Title,Stat Value
0,Name,Matheus Nunes
1,FotMob rating,6.1
2,Minutes played,11
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,8/8 (100%)
7,Chances created,0
8,Expected assists (xA),0.03
9,xG + xA,0.03


,Stat Title,Stat Value
0,Name,Max Kilman
1,FotMob rating,5.5
2,Minutes played,90
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,55/60 (92%)
7,Chances created,0
8,Touches,68
9,Accurate long balls,1/4 (25%)


,Stat Title,Stat Value
0,Name,Matheus Cunha
1,FotMob rating,6.0
2,Minutes played,76
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,25/30 (83%)
7,Chances created,0
8,Expected assists (xA),0.02
9,xG + xA,0.02


,Stat Title,Stat Value
0,Name,Jack Grealish
1,FotMob rating,6.4
2,Minutes played,10
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,7/7 (100%)
7,Chances created,0
8,Expected assists (xA),0.01
9,xG + xA,0.01


,Stat Title,Stat Value
0,Name,Bernardo Silva
1,FotMob rating,6.9
2,Minutes played,80
3,Goals,0
4,Assists,0
5,Total shots,1
6,Accurate passes,26/30 (87%)
7,Chances created,2
8,Expected goals (xG),0.09
9,Expected goals on target (xGOT),0.00


...

,Stat Title,Stat Value
0,Name,Toti Gomes
1,FotMob rating,5.7
2,Minutes played,90
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,48/54 (89%)
7,Chances created,0
8,Touches,67
9,Successful dribbles,2/2 (100%)


,Stat Title,Stat Value
0,Name,Jeremy Doku
1,FotMob rating,6.9
2,Minutes played,10
3,Goals,0
4,Assists,0
5,Total shots,1
6,Accurate passes,9/9 (100%)
7,Chances created,1
8,Expected goals (xG),0.04
9,Expected goals on target (xGOT),0.07


,Stat Title,Stat Value
0,Name,Erling Haaland
1,FotMob rating,9.7
2,Minutes played,82
3,Goals,4
4,Assists,0
5,Total shots,8
6,Accurate passes,10/15 (67%)
7,Chances created,1
8,Penalties won,1
9,Expected goals (xG),2.11


,Stat Title,Stat Value
0,Name,Mario Lemina
1,FotMob rating,6.6
2,Minutes played,45
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,29/33 (88%)
7,Chances created,0
8,Touches,38
9,Successful dribbles,1/1 (100%)


,Stat Title,Stat Value
0,Name,Pablo Sarabia
1,FotMob rating,6.6
2,Minutes played,14
3,Goals,0
4,Assists,0
5,Total shots,0
6,Accurate passes,9/14 (64%)
7,Chances created,0
8,Expected goals (xG),0.43
9,Expected goals on target (xGOT),0.00


In [90]:
# Importing some packages to help us retry after reaching rate limits

import time
from googleapiclient.errors import HttpError

# Repeating the process of accessing our Google sheet

sheet_id = """Enter your Sheet ID"""
sheet = client.open_by_key(sheet_id).sheet1  


# The sheet data has an additional column for player positions which we do not need to access
# hence the additional steps in dropping that column
data = sheet.get_all_values()
headers = data[0][:-1]
data_rows = [row[:-1] for row in data[1:]] 
sheet_df = pd.DataFrame(data_rows, columns=headers)

# We define a function that will update cells and follow retry logic when the rate limits are reached
def update_cell_with_retry(sheet, row, col, value, max_retries=5, backoff_factor=2):
    """Updates a cell with retry logic to handle quota exceeded errors."""
    for attempt in range(1,10):
        try:
            sheet.update_cell(row, col, value)
            break 
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429:  
                wait_time = 6 * attempt
                print(f"Quota exceeded. Waiting for {wait_time} seconds before retrying.")
                time.sleep(wait_time)
            else:
                raise 
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            raise

# We define a function to update the full sheet by matching the player name and their corresponding stat values
def update_google_sheet(sheet, sheet_df, player_dataframes):
    
    for df in player_dataframes:
        name = df[df['Stat Title'] == 'Name']['Stat Value'].iloc[0]
        
        try:
            
            row_index = sheet_df[sheet_df['Player Name'] == name].index[0] + 2

            for _, row in df.iterrows():
                if row['Stat Title'] in headers:
                    col_index = headers.index(row['Stat Title']) + 1
                    update_cell_with_retry(sheet, row_index, col_index, row['Stat Value'])
        except Exception as e:
            print("Exception: " + e)


update_google_sheet(sheet, sheet_df, player_dataframes)

Quota exceeded. Waiting for 6 seconds before retrying.
Quota exceeded. Waiting for 12 seconds before retrying.
Quota exceeded. Waiting for 18 seconds before retrying.
Quota exceeded. Waiting for 24 seconds before retrying.
Quota exceeded. Waiting for 6 seconds before retrying.
Quota exceeded. Waiting for 12 seconds before retrying.


In [91]:
print("Match stats updated!")

Match stats updated!
